In [1]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline  
import math
import random

In [2]:
# Fonction de normalisation des données 
def normalisation(dff) :
    df = dff.copy()
    for i in df.columns :
        mi = min(df[i])
        ma = max(df[i])
        df[i] =  (df[i]- mi) / (ma - mi)
    
    return df

def createGaussianDataFrame(center,sigma,nb_points):
    return pd.DataFrame(np.random.multivariate_normal(center,sigma,nb_points))

def createXOR(nb_points,var) :
    G1 = createGaussianDataFrame(np.array([1,1]),np.array([[var,0],[0,var]]),nb_points)
    G2 = createGaussianDataFrame(np.array([0,0]),np.array([[var,0],[0,var]]),nb_points)
    G3 = createGaussianDataFrame(np.array([0,1]),np.array([[var,0],[0,var]]),nb_points)
    G4 = createGaussianDataFrame(np.array([1,0]),np.array([[var,0],[0,var]]),nb_points)
    return G1.append(G2.append(G3.append(G4,ignore_index=True),ignore_index=True),ignore_index = True)

def createParal(nb_points,var1,var2) :
    G1 = createGaussianDataFrame(np.array([1,0.6]),np.array([[var1,var2],[var2,var1]]),nb_points)
    G2 = createGaussianDataFrame(np.array([1,0.4]),np.array([[var1,var2],[var2,var1]]),nb_points)
    return G1.append(G2,ignore_index = True)

In [3]:
# Fonctions utiles

# Fonctions qui renvoie les combinaisons possible entre les valeurs de deux listes
def combliste(l1,l2):
    res = []
    for e1 in l1 :
        for e2 in l2 :
            res.append(e1+e2)        
    return res

# Fonctions qui renvoie les combinaisons possible entre les valeurs de plusieurs listes
def comblistes(ensListes):
    res = ensListes[0]
    for i in range(len(ensListes) - 1) :
        res = combliste(res,ensListes[i + 1])
    resultat = []
    for e in res :
        e = np.array(e)
        resultat.append(e)
    for i in range(len(resultat)) :
        resultat[i] = resultat[i].reshape(len(resultat[i])/2,2)
    return resultat

# Fonctions qui renvoie les listes des intervalles pour chaque attribut (dimension) 
def getIntervalles(array_Bornes) :
    res = []
    for i in range(len(array_Bornes[0])) :
        colonne = list(array_Bornes[0:,i])
        col = []
        for v in range(len(colonne) - 1) :
            col.append([colonne[v],colonne[v+1]])
        res.append(col)
    return res
 
# Fonction qui renvoie l'ensemble des partie d'une liste    
def partiesliste(seq,dim):
    p = []
    i, imax = 0, 2**len(seq)-1
    while i <= imax:
        s = []
        j, jmax = 0, len(seq)-1
        while j <= jmax:
            if (i>>j)&1 == 1:
                s.append(seq[j])
            j += 1
        p.append(s)
        i += 1 
    del p[0]
    resultat = []
    for i in range(dim) :
        for e in p :
            if(len(e) == i +1 ) :
                resultat.append(e)
    
    return resultat           
        

In [4]:
print(partiesliste([1,2,3,4],4))

[[1], [2], [3], [4], [1, 2], [1, 3], [2, 3], [1, 4], [2, 4], [3, 4], [1, 2, 3], [1, 2, 4], [1, 3, 4], [2, 3, 4], [1, 2, 3, 4]]


In [5]:
data = createXOR(10,0.01)

In [24]:
# - - Classe pour représenter une cellule (units)
class unit :
    def __init__(self,dim,dNames,id) :
        """ d : Nombre de dimensions
            dNames : Noms des dimensions (Attributs)
            lh_Array : Tableau représentant le début et la fin de l'interavalle pour chaque attribut  
        """
        self.dim = dim 
        self.dNames = dNames
        self.lh_Array = []
        self.id = id
        self.dense = False
        self.nuCluster = -1
        self.connectedCells = []
        
    # revoie les noms des attributs
    def getNames (self) :
        return self.dNames
    
    # revoie le nom de l'attributs indice
    def getNames (self,indice) :
        return self.dNames[indice]
    
    # revoie le nombre de dimensions
    def getDimension():
        return self.dim
    
    # revoie le début  et la fin de l'intervalle de la dimension indice
    def getLH(self,indice) :
        return self.lh_Array[indice]
    
    # rajoute le début  et la fin de l'intervalle de la dimension indice
    def addLH(self,lh) :
        self.lh_Array.append(lh)
    
    # affichage d'une cellule 
    def display(self) :
        print("cellule id : ",self.id)
        for ligne in self.lh_Array : 
            print("intervalle : ",ligne)
        print("\n density : ",self.dense)
    
    # Fonction qui renvoie true si un point appartient a la cellule pour un ensemble de dimensions , False sinon
    def contains(self,x,dim) :
        for d in dim :
            if(x[d] >= self.lh_Array[0][d][1] or x[d] < self.lh_Array[0][d][0]):
                return False
        return True
    
    # Fonction qui renvoie true si la cellule est dense pour un ensemble de dimensions, False sinon
    def is_a_dense_cell(self,base,dim,taux) :
        nb_in = 0
        for x in base.values :
            if(self.contains(x,dim)):
                nb_in += 1
        return ( nb_in / (len(base) * 1.0) ) > taux
    
    # Fonction qui renvoie true si la cellule a au moins une face en commun avec la cellule donner en paramétre
    # pour un ensemble de dimension, False sinon
    def has_commonFace_with(self,cell,dim):
        # Nombre d'intervalles différents
        if(not cell.dense) : return False
        nb_dif = 0
        
        # htk = l'tk or h'tk = ltk
        dif = False
        
        for d in dim :
            if(self.lh_Array[0][d][0] != cell.lh_Array[0][d][0] or self.lh_Array[0][d][1] != cell.lh_Array[0][d][1]) :
                nb_dif += 1
                if( nb_dif > 1 ) :# Si le nombre d'intervalles différents est superieur a 1 on renvoie False
                    return False
                else : # Sinon on verifie que l'intervalle en question relie les deux cellules
                    dif = ((self.lh_Array[0][d][1] == cell.lh_Array[0][d][0] or  self.lh_Array[0][d][0] == cell.lh_Array[0][d][1])) 
        return dif          
    # Fonction qui renvoie l'ensemble des element qui ont une face en commun avec la cellule
    def all_commonFaces(self,dim,cellules ):
        res = []
        for c in cellules :
            if(self.has_commonFace_with(c,dim)):
                res.append(c)
        return res
    
    # Fonction qui renvoie true si la cellule a au moins une face en commun avec la cellule donner en paramétre
    # pour un ensemble de dimension, False sinon
    def is_connected_with(self,cell,dim,cellules ):
        cellules_copy = list(cellules)
        
        if(not cell.dense) : return False   
        if(self.has_commonFace_with(cell,dim) or self.id == cell.id) : return True
        for c in cell.all_commonFaces(dim,cellules) :
            if(self.has_commonFace_with(c,dim)): return True
        if(len(cellules_copy) == 0 ) : return False
        
        for c in cellules_copy :
            if(self.has_commonFace_with(c,dim) and cell.has_commonFace_with(c,dim)) : return True
        
        c = cellules_copy.pop(0)
        if(self.is_connected_with(c,dim,cellules_copy) and cell.is_connected_with(c,dim,cellules_copy) ) :
            return True
        return False
    def all_common_faces(self,dim,cellules ):
        res = []
        for c in cellules :
            if(self.has_commonFace_with(c,dim)):
                res.append(c)
        return res
    
    # Fonction qui renvoie une liste contenant tout les id des cellules avec lesquelles elle est connectée
    # pour un ensemble de dimension
    def all_connected(self,dim,cellules) :
        res =[] 
        for c in cellules :
            if(self.is_connected_with(c,dim,cellules)) :
                res.append(c.id)
        return res
    
    # Fonction qui renvoie l'ensemble des points contenus dans une cellules pour un ensemble de dimensions donné
    
    def get_points(self,dim,base) :
        res = []
        for i,x in base.iterrows() :
            if(self.contains(x,dim)) :
                res.append(i)
        return res
            

In [ ]:
for j,x in base.iterrows():
            centroideProche = plus_proche(fdist,pd.DataFrame(x).T,centroides)
            d[centroideProche].append(j)
    return d

In [58]:
def centroide(df) :
    return pd.DataFrame(np.mean(df)).T

In [249]:


# - - Classe pour représenter une grille de cellules (Grid)
class grid :
    def __init__(self,base,nb_intervalles,taux) :
        """ base : DataFrame contenant notre ensemble de données 
        """
        self.base = base
        self.taux = taux
        self.nb_intervalles = nb_intervalles
        self.dim = len(base.max())
        # Noms des dimensions (attributs)
        self.dNames = [i for i in self.base.columns]
    # Renvoie la base de données 
    def getBase (self) :
        return self.base 
    
    # Renvoie le nom de l'attributs indice
    def createGrid (self) :
        # Tableaux contennat la taille de chaque intervalles pour les dimensions
        l_array = self.base.max()
        h_array = self.base.min()
        intervalles = ( l_array - h_array )   / self.nb_intervalles 
        
        # Grille contenant nb_intervalles^dim cellules
        self.grille = []
        
        # Array contenant toutes les valeurs successives de chaque attribut dans chaque intervalle (dimension) 
        array_bornes = np.array([list(h_array + intervalles * i) for i in range(self.nb_intervalles +1)])
        
        # Liste contenant toutes les valeurs d'intervalles prises par chacune des cellules
        cell_values = comblistes(getIntervalles(array_bornes))
        
        for i in range(self.nb_intervalles**self.dim) :    
                u = unit(self.dim,self.dNames,i)
                u.addLH(cell_values[i])
                self.grille.append(u)
        c = self.get_clusters([0,1])
        for cc,v in c.items() :
            print("cluster : ",cc,v)
    
    # Fonction qui marque toutes les cellules dense pour un certaint taux et pour un ensemble de dimensions comme danse 
    def mark_cells(self,dim) :
        for c in self.grille :
            if(c.is_a_dense_cell(self.base,dim,self.taux)) :
                c.dense = True  
    
    # Fonction qui renvoie les Clusters et leurs Cellules pour un ensemble de dimensions donné
    def get_clusters(self,dim) :
        clusters = dict()
        # On marque toutes les cellules dense 
        self.mark_cells(dim)
        # Liste des cellules marquées 
        cells_marked = []
        for c in self.grille :
            if(c.dense == True ) : 
                cells_marked.append(c)
        #print("Les cellules denses sont : ",[c.id for c in cells_marked])
        key = 0
        while(len(cells_marked) > 1 ):
            r =cells_marked[0].all_connected(dim,self.grille)
            print((cells_marked[0]).id)
            del cells_marked[0]
            cell = cells_marked
            cells_marked =[]
            clusters[key] = r 
            #print("clusterkey : ",key, r)
            for e in cell :
                if(e.id not in r ):
                    cells_marked.append(e)      
            key +=1
        return clusters
   
    # Fonction qui renvoie toutes les cellules dont les id sont dans la liste donnée en parametre 
    def get_cells(self,list_id) :
        res = []
        for c in self.grille :
            if(c.id in list_id):
                res.append(c)
        return res
    
    # Fonction qui renvoie l'ensemble des points pour un cluster  (a partir de ses cellules ) pour un ensemble de dimensions donné
    def get_cluster_points(self,dim,cells_ids) :
        cells = self.get_cells(cells_ids)
        points = []
        for i in range(len(cells)) :
            points += cells[i].get_points(dim,self.base)
        df = pd.DataFrame(self.base.iloc[points[0:]])
        #print("centroide : " ,centroide(df))
        return points, centroide(df)

    # Fonction qui renvoie l'ensemble des points pour tous les cluster  (a partir de leurs cellules ) pour un ensemble de dimensions donné
    def get_all_clusters_points(self,dim,dict_clusters_id_cells) :
        clusters_points = {}
        centroides = pd.DataFrame()
        for c,v in dict_clusters_id_cells.items() :
            clusters_points[c],centroide = self.get_cluster_points(dim,v)
            centroides = centroides.append(centroide,ignore_index=True)
        dim_name =""
        for d in dim :
            dim_name += str(d)
        #print("centroides : ",centroides)
        return clusters_points,dim_name,centroides
    
    # Fonction qui renvoie un dictionnaire de dictionnaires, 
    # chaque dictionnaire contient tout les clusters pour un certain ensemble de dimensions
    def get_all_clusters_all_dim(self) :
        dict_all_clust_all_dim = {}
        dict_all_centroides_all_dim = {}
        for dim in partiesliste([i for i in range(self.dim)],self.dim) :
            # Récupération de tout les clusters pour dim (cellules)
            dict_clusters_id_cells = self.get_clusters(dim)
            # Récupération de tout les clusters pour dim (points)
            value,name,dict_centroides = self.get_all_clusters_points(dim,dict_clusters_id_cells)
            dict_all_clust_all_dim[name] = value
            dict_all_centroides_all_dim[name] = dict_centroides
        return dict_all_clust_all_dim,dict_all_centroides_all_dim
    
    
    
    # Renvoie le nombre de dimensionscell_values
    def getDimension(self) :
        return self.dim
    
    # Renvoie la grille 
    def geGrid(self) :
        return self.grille

    # Affichage d'une grille
    def display(self) :
        print("############### GRID ##################\n")
        for cell in self.grille :
            cell.display()

In [275]:
import matplotlib
import colorsys

def AffichagesClusters(centroides,dictAffectation,base):
    
    M_data2D = base.as_matrix()
    
    colonne_X= M_data2D[0:,0]
    
    colonne_Y= M_data2D[0:,1]
    
    #Nombre de couluers
    colorNbr = len(centroides)
    print("nb_color = " , colorNbr )
    #Choix aléatoire des couleurs
    colorNames = list(matplotlib.colors.cnames.keys())
    colors = [i for i in range(len(colorNames))]
    random.shuffle(colors)
    colors =colors[0:colorNbr] 
    print("color = ",colors)
    
    
    colorMap = [] 
    for i in range(colorNbr) :
        colorMap.append(colorNames[colors[i]])
    colorMap = np.array(colorMap)
    
    categories = np.zeros(len(base))
    categories = np.array(categories)
    for key,values in dictAffectation.items() :
        for indice in values :
            categories[indice] = int(key)
    categories = categories.astype(int)
    
    plt.scatter(colonne_X,colonne_Y,s=100,c=colorMap[categories.astype(int)])
    
    #Affichage des centroides finaux en noir 
    M_data2D = centroides.as_matrix()
    colonne_X= M_data2D[0:,0]
    colonne_Y= M_data2D[0:,1]
    plt.scatter(colonne_X,colonne_Y,color='black')
# Fonction d'affichage des résultats
def affichageResults(les_centres, l_affectation,nb_iter,convergence) :
    print("Le nombre d'itération : ",nb_iter)
    print("convergence : ", convergence)
    print("Les centres : \n",les_centres)
    print("Les affectations : \n")

    for c,v in l_affectation.items() :
        print("Cluster",c," : ",v)

    AffichagesClusters(les_centres, l_affectation,points)

In [285]:
data = createXOR(1000,0.01)
len(data)

4000

In [286]:
g =grid(data,5,0.05)
g.createGrid()

0
3
15
18
('cluster : ', 0, [0, 1, 5, 6])
('cluster : ', 1, [3, 4, 8, 9])
('cluster : ', 2, [15, 16, 20, 21])
('cluster : ', 3, [18, 23, 24])


In [281]:

res,resCentr = g.get_all_clusters_all_dim()

0
15
(' cluster : ', 0, [10001, 10004, 10005, 10006, 10012, 10013, 10014, 10016, 10021, 10023, 10026, 10028, 10030, 10033, 10035, 10038, 10039, 10041, 10042, 10045, 10050, 10051, 10052, 10053, 10056, 10057, 10058, 10063, 10064, 10068, 10069, 10072, 10073, 10076, 10077, 10078, 10079, 10080, 10081, 10082, 10088, 10089, 10090, 10093, 10096, 10097, 10098, 10099, 10102, 10104, 10105, 10107, 10112, 10115, 10117, 10118, 10119, 10121, 10124, 10130, 10131, 10132, 10135, 10136, 10137, 10139, 10140, 10142, 10145, 10149, 10150, 10154, 10155, 10159, 10162, 10163, 10164, 10168, 10170, 10172, 10174, 10175, 10177, 10178, 10181, 10183, 10186, 10187, 10188, 10189, 10191, 10192, 10194, 10197, 10198, 10204, 10205, 10207, 10208, 10209, 10210, 10215, 10216, 10218, 10220, 10222, 10223, 10225, 10226, 10229, 10233, 10235, 10237, 10238, 10240, 10241, 10245, 10246, 10248, 10249, 10250, 10251, 10254, 10256, 10259, 10260, 10261, 10267, 10271, 10276, 10283, 10285, 10287, 10289, 10292, 10296, 10297, 10299, 10300, 10

KeyboardInterrupt: 

In [287]:
e = g.get_clusters([0,1])

0
3
15
18


In [288]:
g.get_all_clusters_points([0,1],e)
#for c,v in clusters_points.items() :
            #print(" cluster : " ,c,v)

(' cluster : ', 0, [1007, 1015, 1018, 1022, 1026, 1032, 1034, 1038, 1040, 1047, 1051, 1055, 1057, 1061, 1065, 1069, 1075, 1080, 1082, 1090, 1094, 1096, 1100, 1110, 1112, 1114, 1122, 1124, 1126, 1127, 1129, 1132, 1133, 1134, 1139, 1142, 1145, 1148, 1149, 1151, 1154, 1160, 1167, 1169, 1177, 1188, 1189, 1194, 1197, 1199, 1202, 1203, 1209, 1211, 1213, 1216, 1218, 1219, 1224, 1226, 1227, 1229, 1236, 1240, 1242, 1247, 1256, 1258, 1262, 1267, 1270, 1272, 1274, 1276, 1280, 1282, 1288, 1292, 1300, 1302, 1305, 1306, 1310, 1312, 1317, 1324, 1325, 1335, 1336, 1342, 1346, 1348, 1356, 1359, 1360, 1368, 1370, 1371, 1374, 1376, 1379, 1381, 1387, 1390, 1392, 1395, 1396, 1398, 1402, 1404, 1405, 1413, 1417, 1418, 1420, 1423, 1430, 1445, 1446, 1447, 1448, 1454, 1456, 1459, 1462, 1476, 1481, 1492, 1493, 1500, 1504, 1506, 1509, 1520, 1521, 1523, 1524, 1525, 1527, 1530, 1541, 1542, 1546, 1548, 1551, 1553, 1554, 1565, 1569, 1572, 1573, 1574, 1578, 1584, 1586, 1587, 1592, 1597, 1599, 1604, 1608, 1609, 1612, 16

({0: [1007,
   1015,
   1018,
   1022,
   1026,
   1032,
   1034,
   1038,
   1040,
   1047,
   1051,
   1055,
   1057,
   1061,
   1065,
   1069,
   1075,
   1080,
   1082,
   1090,
   1094,
   1096,
   1100,
   1110,
   1112,
   1114,
   1122,
   1124,
   1126,
   1127,
   1129,
   1132,
   1133,
   1134,
   1139,
   1142,
   1145,
   1148,
   1149,
   1151,
   1154,
   1160,
   1167,
   1169,
   1177,
   1188,
   1189,
   1194,
   1197,
   1199,
   1202,
   1203,
   1209,
   1211,
   1213,
   1216,
   1218,
   1219,
   1224,
   1226,
   1227,
   1229,
   1236,
   1240,
   1242,
   1247,
   1256,
   1258,
   1262,
   1267,
   1270,
   1272,
   1274,
   1276,
   1280,
   1282,
   1288,
   1292,
   1300,
   1302,
   1305,
   1306,
   1310,
   1312,
   1317,
   1324,
   1325,
   1335,
   1336,
   1342,
   1346,
   1348,
   1356,
   1359,
   1360,
   1368,
   1370,
   1371,
   1374,
   1376,
   1379,
   1381,
   1387,
   1390,
   1392,
   1395,
   1396,
   1398,
   1402,
   1404,
   1405

In [289]:
print(len(res['01']))

3


In [293]:
AffichagesClusters(resCentr['01'],e,data)

('nb_color = ', 3)
('color = ', [62, 78, 15])


IndexError: index 3 is out of bounds for size 3